In [5]:
from keras.layers import Dense, Flatten
from keras.models import Sequential
from keras.callbacks import Callback
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import keras
import subprocess
import os

import wandb
from wandb.keras import WandbCallback

In [17]:
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten

In [6]:
def load_fer2013():
    if not os.path.exists("fer2013"):
        print("Downloading the face emotion dataset...")
        subprocess.check_output("curl -SL https://www.dropbox.com/s/opuvvdv3uligypx/fer2013.tar | tar xz", shell=True)
    data = pd.read_csv("fer2013/fer2013.csv")
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = np.asarray(pixel_sequence.split(' '), dtype=np.uint8).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), (width, height))
        faces.append(face.astype('float32'))

    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()

    val_faces = faces[int(len(faces) * 0.8):]
    val_emotions = emotions[int(len(faces) * 0.8):]
    train_faces = faces[:int(len(faces) * 0.8)]
    train_emotions = emotions[:int(len(faces) * 0.8)]
    
    return train_faces, train_emotions, val_faces, val_emotions


In [25]:
# loading dataset

train_faces, train_emotions, val_faces, val_emotions = load_fer2013()
num_samples, num_classes = train_emotions.shape

train_faces /= 255.
val_faces /= 255.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


In [22]:
train_faces.shape

(28709, 48, 48, 1)

In [77]:
config = run.config

config.batch_size = 32
#config.num_epochs = 20

input_shape = (48, 48, 1)

In [78]:
#Added from cnn.py
#config = run.config
config.first_layer_convs = 32
config.first_layer_conv_width = 3
config.first_layer_conv_height = 3
#config.dropout = 0.2
config.dense_layer_size = 128
#config.img_width = 28
#config.img_height = 28
config.num_epochs = 32

In [79]:
run = wandb.init()

W&B Run: https://app.wandb.ai/qualcomm/emotion-aug21/runs/3m7nbfcg
Wrap your training loop with `with wandb.monitor():` to display live results.


In [80]:
# Testing
#config.num_epochs = 32

model = Sequential()

model.add(Conv2D(32,
    (config.first_layer_conv_width, config.first_layer_conv_height),
    input_shape=(48, 48,1),
    padding='valid',
    activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(32,
    (config.first_layer_conv_width, config.first_layer_conv_height),
    #input_shape=(48, 48,1),
    padding='valid',
    activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(32,
    (config.first_layer_conv_width, config.first_layer_conv_height),
    #input_shape=(48, 48,1),
    padding='valid',
    activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(32,
    (config.first_layer_conv_width, config.first_layer_conv_height),
    #input_shape=(48, 48,1),
    padding='valid',
    activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(32,
    (config.first_layer_conv_width, config.first_layer_conv_height),
    #input_shape=(48, 48,1),
    padding='valid',
    activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))


model.add(Flatten(input_shape=input_shape))
model.add(Dense(config.dense_layer_size, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation="softmax"))

model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])

#model.load_weights("emotion.h5")

model.fit(train_faces, train_emotions, batch_size=config.batch_size,
        epochs=config.num_epochs, verbose=1, callbacks=[
            WandbCallback(data_type="image", labels=["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"])
        ], validation_data=(val_faces, val_emotions))


Train on 28709 samples, validate on 7178 samples
Epoch 1/32
28709/28709 [==============================] - 24s 852us/step - loss: 1.8038 - acc: 0.2544 - val_loss: 1.8519 - val_acc: 0.2282
Epoch 2/32
28709/28709 [==============================] - 22s 775us/step - loss: 1.7336 - acc: 0.3005 - val_loss: 1.8542 - val_acc: 0.2458
Epoch 3/32
28709/28709 [==============================] - 22s 777us/step - loss: 1.6785 - acc: 0.3328 - val_loss: 1.7227 - val_acc: 0.3123
Epoch 4/32
28709/28709 [==============================] - 22s 778us/step - loss: 1.6405 - acc: 0.3519 - val_loss: 1.6534 - val_acc: 0.3381
Epoch 5/32
28709/28709 [==============================] - 22s 781us/step - loss: 1.6042 - acc: 0.3739 - val_loss: 1.6533 - val_acc: 0.3452
Epoch 6/32
28709/28709 [==============================] - 23s 785us/step - loss: 1.5756 - acc: 0.3844 - val_loss: 1.5566 - val_acc: 0.3929
Epoch 7/32
28709/28709 [==============================] - 22s 780us/step - loss: 1.5564 - acc: 0.3913 - val_loss: 1.5

In [15]:
model.save("emotion.h5")